# How To: Run SModelS as a python library

In [1]:
# Set up the path to SModelS installation folder
import sys; sys.path.append("."); import smodels_paths

In [2]:
from smodels.tools import runtime
from smodels.theory import decomposer
from smodels.tools.physicsUnits import fb, GeV, TeV
from smodels.theory.theoryPrediction import theoryPredictionsFor
from smodels.experiment.databaseObj import Database
from smodels.tools import coverage
from smodels.tools.smodelsLogging import setLogLevel
from smodels.particlesLoader import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
setLogLevel("info")

### Main input:

In [3]:
# Path to input file (either a SLHA or LHE file)
slhafile = 'inputFiles/slha/lightEWinos.slha'
#Define your model
runtime.modelFile = 'smodels.share.models.mssm' 
model = Model(BSMparticles=BSMList, SMparticles=SMList)
model.updateParticles(inputFile=slhafile)

### Decompose the input model:

In [4]:
# Set main options for decomposition
sigmacut = 0.01 * fb
mingap = 5. * GeV
# Decompose model (use slhaDecomposer for SLHA input or lheDecomposer for LHE input)
toplist = decomposer.decompose(model, sigmacut, doCompress=True, doInvisible=True, minmassgap=mingap)

# Access basic information from decomposition, using the topology list and topology objects:
print( "\n Decomposition Results: " )
print( "\t  Total number of topologies: %i " %len(toplist) )
nel = sum([len(top.elementList) for top in toplist])
print( "\t  Total number of elements = %i " %nel )


 Decomposition Results: 
	  Total number of topologies: 51 
	  Total number of elements = 14985 


### Load the Database of experimental results:

In [5]:
# Set the path to the database
database = Database("official")
# Load the experimental results to be used.
# In this case, all results are employed.
listOfExpRes = database.getExpResults()

# Print basic information about the results loaded.
# Count the number of loaded UL and EM experimental results:
nUL, nEM = 0, 0
for exp in listOfExpRes:
    expType = exp.getValuesFor('dataType')[0]
    if expType == 'upperLimit':
        nUL += 1
    elif  expType == 'efficiencyMap':
        nEM += 1
print("\n Loaded Database with %i UL results and %i EM results " %(nUL,nEM))

INFO in databaseObj.loadBinaryFile() in 177: loading binary db file /home/lessa/.cache/smodels/official200rc4.pcl format version 207
INFO in databaseObj.loadBinaryFile() in 184: Loaded database from /home/lessa/.cache/smodels/official200rc4.pcl in 0.6 secs.



 Loaded Database with 55 UL results and 21 EM results 


### Match the decomposed simplified models with the experimental database of constraints:

In [6]:
# Compute the theory predictions for each experimental result and print them:
print("\n Theory Predictions and Constraints:")
rmax = 0.
bestResult = None
for expResult in listOfExpRes:
    predictions = theoryPredictionsFor(expResult, toplist)
    if not predictions: continue # Skip if there are no constraints from this result
    print('\n %s (%i TeV)' %(expResult.globalInfo.id,expResult.globalInfo.sqrts.asNumber(TeV)))
    for theoryPrediction in predictions:
        dataset = theoryPrediction.dataset
        datasetID = theoryPrediction.dataId()
        mass = theoryPrediction.mass
        txnames = [str(txname) for txname in theoryPrediction.txnames]
        PIDs =  theoryPrediction.PIDs         
        print( "------------------------" )
        print( "TxNames = ",txnames )  
        print( "Theory Prediction = ",theoryPrediction.xsection.value )  #Signal cross section
        # Get the corresponding upper limit:
        print( "UL for theory prediction = ",theoryPrediction.upperLimit )
        # Compute the r-value
        r = theoryPrediction.getRValue()
        print( "r = ",r )
        #Compute likelihhod and chi^2 for EM-type results:
        if dataset.dataInfo.dataType == 'efficiencyMap':
            theoryPrediction.computeStatistics()
            print( 'Chi2, likelihood=', theoryPrediction.chi2, theoryPrediction.likelihood )
        if r > rmax:
            rmax = r
            bestResult = expResult.globalInfo.id

# Print the most constraining experimental result
print( "\nThe largest r-value (theory/upper limit ratio) is ",rmax )
if rmax > 1.:
    print( "(The input model is likely excluded by %s)" %bestResult )
else:
    print( "(The input model is not excluded by the simplified model results)" )


 Theory Predictions and Constraints:

 ATLAS-SUSY-2015-06 (13 TeV)
------------------------
TxNames =  ['T1']
Theory Prediction =  5.19E-06 [pb]
UL for theory prediction =  1.79E+00 [fb]
r =  0.0029013935307768326
Chi2, likelihood= 2.3776849368423356 0.007169156710956845

 CMS-PAS-SUS-17-004 (13 TeV)
------------------------
TxNames =  ['TChiWH']
Theory Prediction =  6.10E-04 [pb]
UL for theory prediction =  3.82E+02 [fb]
r =  0.0015950037773415759
------------------------
TxNames =  ['TChiWZ']
Theory Prediction =  2.79E-04 [pb]
UL for theory prediction =  1.18E+02 [fb]
r =  0.00236222889582526

 CMS-SUS-16-033 (13 TeV)
------------------------
TxNames =  ['T2']
Theory Prediction =  2.54E-05 [pb]
UL for theory prediction =  3.60E+01 [fb]
r =  0.0007039737677262512

 CMS-SUS-16-034 (13 TeV)
------------------------
TxNames =  ['TChiWZ']
Theory Prediction =  2.79E-04 [pb]
UL for theory prediction =  2.32E+02 [fb]
r =  0.001203334064103375

 CMS-SUS-16-036 (13 TeV)
----------------------


 CMS-SUS-13-007 (8 TeV)
------------------------
TxNames =  ['T1tttt']
Theory Prediction =  2.92E-06 [pb]
UL for theory prediction =  1.56E-01 [fb]
r =  0.01873521113129384
Chi2, likelihood= 3.745224644243316 0.06337106075103033

 CMS-SUS-13-012 (8 TeV)
------------------------
TxNames =  ['T2']
Theory Prediction =  3.82E-06 [pb]
UL for theory prediction =  7.57E+01 [fb]
r =  5.0492342410963783e-05
------------------------
TxNames =  ['T1']
Theory Prediction =  2.03E-04 [pb]
UL for theory prediction =  5.63E+01 [fb]
r =  0.0035971001930282867
------------------------
TxNames =  ['T1tttt']
Theory Prediction =  6.70E-04 [pb]
UL for theory prediction =  9.91E+01 [fb]
r =  0.006760837896922529

 CMS-SUS-13-012 (8 TeV)
------------------------
TxNames =  ['TChiWW', 'T2', 'TChiWZ', 'T1bbbb', 'T1tttt', 'T1', 'T5WWoff']
Theory Prediction =  1.12E-04 [pb]
UL for theory prediction =  1.22E+00 [fb]
r =  0.09203994368334056
Chi2, likelihood= 0.18732678715205608 0.0034823991276502326

 CMS-SUS-13-

### Check for simplified models in the input model which were not tested by the Database:

In [7]:
#Find out missing topologies for sqrts=8*TeV:
uncovered = coverage.Uncovered(toplist,sigmacut,sqrts=8.*TeV)
#Print uncovered cross-sections:
print("\nTotal missing topology cross section (fb): %10.3E\n" %(uncovered.missingTopos.getTotalXsec()))
print("Total cross section where we are outside the mass grid (fb): %10.3E\n" %(uncovered.outsideGrid.getTotalXsec()))
print("Total cross section with longlived decays (fb): %10.3E\n" %(uncovered.longLived.getTotalXsec()))
print("Total cross section with displaced decays (fb): %10.3E\n" %(uncovered.displaced.getTotalXsec()))
print("Total cross section with MET decays (fb): %10.3E\n" %(uncovered.MET.getTotalXsec()))

#Print some of the missing topologies:
if uncovered.missingTopos.generalElements:
    print('Missing topologies (up to 3):' )
    for genEl in sorted(uncovered.missingTopos.generalElements, key=lambda x: x.missingX, reverse=True)[:3]:
        print('Element:', str(genEl._outputDescription))
        print('\tcross-section (fb):', genEl.missingX)
else:
    print("No missing topologies found\n")

#Print elements with displaced decays:
if uncovered.displaced.generalElements:
    print('\nElements with displaced vertices (up to 2):' )    
    for genEl in sorted(uncovered.displaced.generalElements, key=lambda x: x.missingX, reverse=True)[:2]:
        print('Element:', str(genEl._outputDescription))
        print('\tcross-section (fb):', genEl.missingX)
else:
    print("\nNo displaced decays")



Total missing topology cross section (fb):  3.717E+03

Total cross section where we are outside the mass grid (fb):  9.984E+01

Total cross section with longlived decays (fb):  0.000E+00

Total cross section with displaced decays (fb):  0.000E+00

Total cross section with MET decays (fb):  3.717E+03

Missing topologies (up to 3):
Element: [[[l,nu]],[[jet,jet]]]  (prompt,prompt)
	cross-section (fb): 644.0092445884676
Element: [[[nu,ta]],[[jet,jet]]]  (prompt,prompt)
	cross-section (fb): 321.1539651599444
Element: [[[b,b]],[[jet,jet]]]  (prompt,prompt)
	cross-section (fb): 274.43218534400495

No displaced decays
